## Sequence Modeling with Hidden Markov Models
In this homework, we will work on sequence data. CpG island detection is a well-known problem in bioinformatics. There are four bases in genomes, A, C, G, and T. In some genomic regions CG bases were observed to be  significantly more common than in other regions. These CG rich regions are called CpG islands. We will not use benchmark data with CpG information. Instead, we will use an HMM model to generate our own data by providing transition and emission probabilties. HMM is a distribution depicted by a graphical model, and it is a generative model. The following diagram shows a 2 state HMM. Emission probabilities are shown in rectangles, and transition probabilities are shown in edge labels. 

<img src="cpg.png" alt="cpg" style="width: 350px;" align="left"/>

### Synthetic Sequence Generation

We define a model and probabilities to generate our own sequence data. The probabilities we chose reflect this biological phenomenon. We will later see that our HMM sequence model will predict the location of CpG islands.


In [ ]:
from collections import namedtuple

import numpy as np
import pomegranate

We define our generator model using the transition and emission probabilities in the figure.

In [ ]:
def build_model(dist1, dist2, trans, name1, name2):
    """
    Args:
        dist1: pomegranate.DiscreteDistribution
            Non island emission probabilities
        dist2: pomegranate.DiscreteDistribution
            CpG island emission probabilities
        trans: collections.namedtuple
            Transition probabilities
        name1: str
            State 1 name
        name2: str
            State 2 name
    """
    s1 = pomegranate.State(dist1, name=name1)
    s2 = pomegranate.State(dist2, name=name2)

    start_s1, start_s2, s11, s12, s21, s22 = trans

    # create HMM model
    model = pomegranate.HiddenMarkovModel()
    # add states
    model.add_states(s1, s2)

    # add transitions
    model.add_transition(model.start, s1, start_s1)
    model.add_transition(model.start, s2, start_s2)
    model.add_transition(s1, s1, s11)
    model.add_transition(s1, s2, s12)
    model.add_transition(s2, s1, s21)
    model.add_transition(s2, s2, s22)

    model.bake()

    return model


non_island = pomegranate.DiscreteDistribution(
    {"A": 0.25, "C": 0.25, "T": 0.25, "G": 0.25})
island = pomegranate.DiscreteDistribution(
    {"A": 0.10, "C": 0.40, "T": 0.10, "G": 0.40})

trans = {"start_s1": 0.5, "start_s2": 0.5,
         "s11": 0.90, "s12": 0.10, "s21": 0.10, "s22": 0.90}
trans = namedtuple("trans", trans.keys())(**trans)

states = ["Non-Island", "CpG Island"]
model = build_model(non_island, island, trans, *states)

We generate synthetic sequence data using the above model.

In [ ]:
def generate_data(model, n, length):
    """
    Samples `n` sequences with `length` from `model`
    """
    return np.array([model.sample(length=length) for i in range(n)])


# We generate 2000 sequences each with length 100
X = generate_data(model, 2000, 100)

We define a new HMM model for CpG island prediction. We will learn the transition and emission probabilities from data.

Note: Training takes about 15 minutes.

In [ ]:
cpg_predictor = pomegranate.HiddenMarkovModel()

# if you have multithread computing resource, you can increase n_jobs
cpg_predictor = cpg_predictor.from_samples(distribution=pomegranate.DiscreteDistribution,
                                           n_components=2, X=X, algorithm="baum-welch",
                                           state_names=states,
                                           verbose=True, n_jobs=1)

In [ ]:
ex_seq = X[3]
# transition and emission probabil
trans, ems = cpg_predictor.forward_backward("".join(ex_seq))

You can see the contents of `trans` and `ems` with `print` statements. `trans` matrix is a square matrix and contains the expected number of transitions across each edge in the model. Each row and each column is a state. Direction of transitions is from rows to columns, i.e., `trans_{ij}` contains the expected number of transitions from state in row_i to state in column_j. We have two states related to our problem, CpG Island and Non-Island states and two other states for start and end of the HMM model. Our `ex_seq` has 100 characters in it, sum of the expected transition counts on each edge (`np.sum(trans)`) is equal to 100. 

`ems` is a 100x2 matrix. Each row is an emission, each column is a state. `ems_{ij}` is the natural logarithm of the probability of emitting `ex_seq[i]` in state `j`. Sum of the emission probabilities of each state for `ex_seq[i]` is 1.

In [ ]:
# print(X[0].shape)
# print(ems.shape)
# print(trans.shape)
# print(trans)
# print(ems)
# np.sum(trans)
# np.sum(np.exp(ems), axis=1) 

Next, we will print the original and predicted state sequences, as well as the probabilities of each state for each emission.

In [ ]:
def predict(model, seq, path, mapping):    
    preds = [state[1].name for state in model.maximum_a_posteriori(seq)[1]]
    preds = [mapping[p] for p in preds]
    path = [mapping[s.name] for s in path]
    
    print("{:10s}: {}".format("Sequence", "".join(seq)))
    print("{:10s}: {}".format("HMM Pred", "".join(map(str, preds))))
    print("{:10s}: {}".format("True Path", "".join(map(str, path))))
    
seq, path = model.sample(n=1, length=100, path=True)[0]

predict(cpg_predictor, seq, path[1:], {"Non-Island": 0, "CpG Island": 1})

### Exercise

This exercise was copied from Dr. Ziv Bar-Joseph lecture notes.

Given model below and the sequence *x*="123412316261636461623411221341", define an HMM model with the given model probabilities to answer the following questions:

1. What is the likelihood of observing sequence *x* given model probabilities? (**Hint: Use forward algorithm**)
2. Is it more likely that the 3rd observed “6” in *x* comes from a Loaded or a Fair die? 
 (**Hint: Use Viterbi algorithm**)
 
You can check the [API documentation](https://pomegranate.readthedocs.io/en/latest/HiddenMarkovModel.html#module-pomegranate.hmm).
 
<img src="casino.png" alt="casino" style="width: 300px;" align="left"/>

<a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by-nc-sa/4.0/88x31.png" /></a><br />This work is licensed under a <a rel="license" href="http://creativecommons.org/licenses/by-nc-sa/4.0/">Creative Commons Attribution-NonCommercial-ShareAlike 4.0 International License</a>.

Please contact Haluk Dogan (<a href="mailto:hdogan@vivaldi.net">hdogan@vivaldi.net</a>) for further questions or inquries.